In [40]:
import cloudscraper, pandas as pd, base64
import subprocess, json

In [41]:
scraper = cloudscraper.create_scraper()

In [42]:
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/'

In [43]:
payload = {"language":"pt-br","pageNumber":1,"pageSize":100,"tradingName":"Petrobras"}
encoded = base64.b64encode(str(payload).encode()).decode()
req = scraper.get(url + encoded)
results = req.json()['results']
df = pd.DataFrame(results)
total_pages = req.json()['page']['totalPages']
for page in range(2, total_pages + 1):
    payload['pageNumber'] = page
    encoded = base64.b64encode(str(payload).encode()).decode()
    req = scraper.get(url + encoded)
    results = req.json()['results']
    df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)

In [44]:
df

,typeStock,dateApproval,valueCash,ratio,corporateAction,lastDatePriorEx,dateClosingPricePriorExDate,closingPricePriorExDate,quotedPerShares,corporateActionPrice,lastDateTimePriorEx
0,ON,09/12/2024,"0,65356508",1,DIVIDENDO,23/12/2024,23/12/2024,"40,02",1,"1,633096",2024-12-23T00:00:00
1,ON,09/12/2024,"0,66410331",1,JRS CAP PROPRIO,23/12/2024,23/12/2024,"40,02",1,"1,659429",2024-12-23T00:00:00
2,ON,09/12/2024,"0,01053822",1,JRS CAP PROPRIO,23/12/2024,23/12/2024,"40,02",1,"0,026332",2024-12-23T00:00:00
3,ON,21/11/2024,"1,55174293",1,DIVIDENDO,11/12/2024,11/12/2024,"43,90",1,"3,534722",2024-12-11T00:00:00
4,ON,08/08/2024,"0,11384838",1,DIVIDENDO,21/08/2024,21/08/2024,"40,73",1,"0,279520",2024-08-21T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
294,PN,24/03/1998,"0,66937024",1000,DIVIDENDO,24/03/1998,24/03/1998,"275,00",1000,"0,243407",1998-03-24T00:00:00
295,PN,21/03/1997,"2,00",1000,DIVIDENDO,21/03/1997,20/03/1997,"221,00",1000,"0,904977",1997-03-21T00:00:00
296,PN,21/03/1997,"2,00",1000,DIVIDENDO,21/03/1997,20/03/1997,"221,00",1000,"0,904977",1997-03-21T00:00:00
297,PN,21/03/1997,"2,1263",1000,DIVIDENDO,21/03/1997,20/03/1997,"221,00",1000,"0,962127",1997-03-21T00:00:00


In [36]:
def execute_on_container(container, company):
    # Cria um dicionário com as variáveis
    variables_dict = {
        "trading_name": company,
    }
    variables = json.dumps(variables_dict)

    # Comando Airflow
    airflow_command = ["docker", "exec", container, "airflow", "dags", "trigger", "-c", variables, "b3_fetch_dividendos"]


    print(f"Executando comando no container {container}")
    print(f"Comando: {' '.join(airflow_command)}")

    try:
        result = subprocess.run(airflow_command, check=True, capture_output=True, text=True)
        print(f"Saída do comando:\n{result.stdout}")
        if result.stderr:
            print(f"Erro:\n{result.stderr}")
        return result
    except subprocess.CalledProcessError as e:
        print(f"Erro ao executar o comando no container {container}:\n{e.stderr}")
        return None

In [37]:
#execute_on_container("airflow-airflow-webserver-1", "Petrobras")

In [38]:
# Agrupar por ano e somar os dividendos
df['valueCash'] = df['valueCash'].str.replace('.', '').str.replace(',', '.').astype(float)
df['dateApproval'] = pd.to_datetime(df['dateApproval'], format="%d/%m/%Y")
df['ano'] = df['dateApproval'].dt.year
dt = df.groupby(['ano', 'typeStock']).agg({'valueCash': 'sum'}).reset_index()


In [39]:
dt

,ano,typeStock,valueCash
0,1996,ON,0.988200
1,1996,PN,5.147100
2,1997,ON,1.439600
3,1997,PN,6.126300
4,1998,ON,1.784200
5,1998,PN,6.152300
6,1999,ON,4.658800
7,1999,PN,6.440800
8,2000,ON,8.160000
9,2000,PN,8.160000
